In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [4]:
items = pd.read_csv('items-translated.csv')
items_real=pd.read_csv('items.csv')
transactions = pd.read_csv('sales_train.csv')
shops = pd.read_csv('shops-translated.csv')
item_cat = pd.read_csv('item_categories-translated.csv')

In [3]:
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
from itertools import product
index_cols = ['shop_id','item_id','date_block_num']
grid = []

blocks = transactions.date_block_num.unique()

for block in blocks:
    curr_shops = transactions[transactions.date_block_num==block]['shop_id'].unique()
    curr_items = transactions[transactions.date_block_num==block]['item_id'].unique()
    grid.append(np.array(list(product(*[curr_shops,curr_items,[block]])),dtype='int32')) 

In [5]:
data=pd.DataFrame(np.vstack(grid),columns=index_cols,dtype='int32')
data=data.sort_values(index_cols)
data.head()

,shop_id,item_id,date_block_num
606857,0,12,1
139255,0,19,0
141495,0,27,0
606011,0,27,1
144968,0,28,0


In [6]:
# gb=transactions.groupby(index_cols)['item_cnt_day'].sum().reset_index()
gb=transactions.groupby(index_cols,as_index=False).agg({'item_price':'median','item_cnt_day':'sum'})
gb.columns=['shop_id','item_id','date_block_num','median_price','item_cnt_month']
gb.head()

,shop_id,item_id,date_block_num,median_price,item_cnt_month
0,0,30,1,265.0,31.0
1,0,31,1,434.0,11.0
2,0,32,0,221.0,6.0
3,0,32,1,221.0,10.0
4,0,33,0,347.0,3.0


In [7]:
all_data=data.merge(gb,on=index_cols,how='left').fillna(0)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [8]:
all_data

,shop_id,item_id,date_block_num,median_price,item_cnt_month
1,0,19,0,0.0,0.0
2,0,27,0,0.0,0.0
4,0,28,0,0.0,0.0
6,0,29,0,0.0,0.0
9,0,32,0,221.0,6.0
...,...,...,...,...,...
10913779,59,22162,33,0.0,0.0
10913782,59,22163,33,0.0,0.0
10913791,59,22164,33,0.0,0.0
10913818,59,22166,33,0.0,0.0


In [9]:
all_data=all_data.merge(shops,on='shop_id',how='left')
all_data=all_data.merge(items,on='item_id',how='left')
all_data=all_data.merge(items_real,on='item_id',how='left')
all_data=all_data.merge(item_cat,on='item_category_id',how='left')

all_data.drop(['item_name'],axis=1,inplace=True)
all_data.head()

,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,Name,item_name_translated,item_category_id,item_category_name_translated
0,0,19,0,0.0,0.0,Yakutsk,Shop,"Ordzhonikidze, 56 francs",/ GOLDEN COLLECTION m / f-72,40,Cinema - DVD
1,0,27,0,0.0,0.0,Yakutsk,Shop,"Ordzhonikidze, 56 francs","007 Legends [PS3, Russian version]",19,Games - PS3
2,0,28,0,0.0,0.0,Yakutsk,Shop,"Ordzhonikidze, 56 francs","007 Legends [PC, Jewel, Russian version]",30,PC Games - Standard Edition
3,0,29,0,0.0,0.0,Yakutsk,Shop,"Ordzhonikidze, 56 francs","007 Legends [Xbox 360, Russian version]",23,Games - XBOX 360
4,0,32,0,221.0,6.0,Yakutsk,Shop,"Ordzhonikidze, 56 francs",1+1,40,Cinema - DVD


In [10]:
all_data.drop(['item_name_translated'],axis=1,inplace=True)
all_data.drop(['Name'],axis=1,inplace=True)

In [11]:
all_data['City']=pd.factorize(all_data.City)[0]
all_data['Type']=pd.factorize(all_data.Type)[0]
all_data.head()

,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,item_category_id,item_category_name_translated
0,0,19,0,0.0,0.0,0,0,40,Cinema - DVD
1,0,27,0,0.0,0.0,0,0,19,Games - PS3
2,0,28,0,0.0,0.0,0,0,30,PC Games - Standard Edition
3,0,29,0,0.0,0.0,0,0,23,Games - XBOX 360
4,0,32,0,221.0,6.0,0,0,40,Cinema - DVD


In [12]:
import nltk

In [13]:
#Getting most common words from category names
all_words=' '.join(all_data.item_category_name_translated.unique())
words=nltk.tokenize.word_tokenize(all_words)
words_dist=nltk.FreqDist(words)
words_dist

FreqDist({'-': 77, 'Games': 15, 'Books': 12, 'Gifts': 12, '(': 12, ')': 12, ',': 10, 'Accessories': 8, 'Game': 8, 'Consoles': 8, ...})

In [14]:
#Most Common Words
words_dist.most_common(20)

[('-', 77),
 ('Games', 15),
 ('Books', 12),
 ('Gifts', 12),
 ('(', 12),
 (')', 12),
 (',', 10),
 ('Accessories', 8),
 ('Game', 8),
 ('Consoles', 8),
 ('Music', 7),
 ('XBOX', 6),
 ('Programs', 6),
 ('Cinema', 5),
 ('PC', 5),
 ('Payment', 5),
 ('of', 4),
 ('cards', 4),
 ('PS3', 3),
 ('Edition', 3)]

In [15]:
all_data['main_cat']=pd.np.where(all_data.item_category_name_translated.str.contains('Games | Game'),'Game',pd.np.where(all_data.item_category_name_translated.str.contains('Books'),'Books',pd.np.where(all_data.item_category_name_translated.str.contains('Gifts'),'Gifts',pd.np.where(all_data.item_category_name_translated.str.contains('Accessories'),'Accessories',pd.np.where(all_data.item_category_name_translated.str.contains('Consoles'),'Consoles',pd.np.where(all_data.item_category_name_translated.str.contains('Music'),'Music',pd.np.where(all_data.item_category_name_translated.str.contains('XBOX'),'XBOX',pd.np.where(all_data.item_category_name_translated.str.contains('Programs'),'Programs',pd.np.where(all_data.item_category_name_translated.str.contains('Cinema'),'Cinema',pd.np.where(all_data.item_category_name_translated.str.contains('PC'),'PC',pd.np.where(all_data.item_category_name_translated.str.contains('Payment'),'Payment','Other')))))))))))
all_data.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,item_category_id,item_category_name_translated,main_cat
0,0,19,0,0.0,0.0,0,0,40,Cinema - DVD,Cinema
1,0,27,0,0.0,0.0,0,0,19,Games - PS3,Game
2,0,28,0,0.0,0.0,0,0,30,PC Games - Standard Edition,Game
3,0,29,0,0.0,0.0,0,0,23,Games - XBOX 360,Game
4,0,32,0,221.0,6.0,0,0,40,Cinema - DVD,Cinema


In [16]:
all_data.drop('item_category_name_translated',axis=1,inplace=True)
all_data['main_cat']=pd.factorize(all_data.main_cat)[0]
all_data.head()

,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,item_category_id,main_cat
0,0,19,0,0.0,0.0,0,0,40,0
1,0,27,0,0.0,0.0,0,0,19,1
2,0,28,0,0.0,0.0,0,0,30,1
3,0,29,0,0.0,0.0,0,0,23,1
4,0,32,0,221.0,6.0,0,0,40,0


In [17]:
get_months=pd.DataFrame(all_data['date_block_num'].unique())
mnths=pd.Series(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']*3)
get_months=pd.concat([get_months,mnths[:34]],axis=1)
get_months.columns=['date_block_num','month']
get_months.head()

,date_block_num,month
0,0,Jan
1,1,Feb
2,2,Mar
3,3,Apr
4,4,May


In [18]:
all_data=all_data.merge(get_months,on='date_block_num',how='left')
all_data

,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,item_category_id,main_cat,month
0,0,19,0,0.0,0.0,0,0,40,0,Jan
1,0,27,0,0.0,0.0,0,0,19,1,Jan
2,0,28,0,0.0,0.0,0,0,30,1,Jan
3,0,29,0,0.0,0.0,0,0,23,1,Jan
4,0,32,0,221.0,6.0,0,0,40,0,Jan
...,...,...,...,...,...,...,...,...,...,...
10913845,59,22162,33,0.0,0.0,24,5,40,0,Oct
10913846,59,22163,33,0.0,0.0,24,5,40,0,Oct
10913847,59,22164,33,0.0,0.0,24,5,37,0,Oct
10913848,59,22166,33,0.0,0.0,24,5,54,3,Oct


In [19]:
all_data['month']=pd.factorize(all_data.month)[0]+1
all_data

,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,item_category_id,main_cat,month
0,0,19,0,0.0,0.0,0,0,40,0,1
1,0,27,0,0.0,0.0,0,0,19,1,1
2,0,28,0,0.0,0.0,0,0,30,1,1
3,0,29,0,0.0,0.0,0,0,23,1,1
4,0,32,0,221.0,6.0,0,0,40,0,1
...,...,...,...,...,...,...,...,...,...,...
10913845,59,22162,33,0.0,0.0,24,5,40,0,10
10913846,59,22163,33,0.0,0.0,24,5,40,0,10
10913847,59,22164,33,0.0,0.0,24,5,37,0,10
10913848,59,22166,33,0.0,0.0,24,5,54,3,10


In [20]:
new_data=all_data.sort_values(['shop_id','item_id','date_block_num'])

In [21]:
new_data.to_csv('new_data.csv',index_label=False)